In [632]:
import pandas as pd
import numpy as np

Loading input into numpy ndarray

In [633]:
def clean_cell(cell: str):
    if cell == "R\n" or "M\n":
        return cell.strip("\n")
    return float(cell)

In [634]:
file = "data/sonar.all-data"
f = open(file)
M = []
for line in f:
    M.append([clean_cell(x) for x in line.split(',')])
M = np.array([np.array(i) for i in M])

Preparing train and val splits

In [635]:
np.random.shuffle(M)
x = M[:, :60].astype(float)
y = M[:, 60]

In [636]:
def normalise_2darray(d2array):
    output_array = []
    for array in d2array:
        x = (array - np.mean(array)) / np.std(array)
        # x[x < 0] *= -1
        # x = (x-np.min(x))/(np.max(x)-np.min(x))
        output_array.append(x)
    return np.asarray(output_array)

x_norm = normalise_2darray(x)
x_norm

array([[-1.0898072 , -1.1012549 , -1.02316525, ..., -1.10493452,
        -1.04933142, -1.08490105],
       [-0.80516528, -0.85757709, -0.82403353, ..., -0.85817608,
        -0.84829271, -0.85218616],
       [-0.83275779, -0.85766521, -0.89127046, ..., -0.94345743,
        -0.95689953, -0.93555031],
       ...,
       [-1.17448994, -1.06767451, -0.97126319, ..., -1.19529814,
        -1.19287052, -1.2015406 ],
       [-0.96814314, -1.00247733, -1.02076401, ..., -1.08719319,
        -1.07562407, -1.08495401],
       [-1.07592463, -1.11308872, -0.97171944, ..., -1.22239486,
        -1.21437908, -1.22385228]])

In [637]:
def encode_classes(target):
    encoded_target = []
    encoding_dict = {'R': 0, 'M': 1}
    
    for x in target:
        encoded_target.append(encoding_dict.get(x))
            
    return np.asarray(encoded_target), encoding_dict

y_enc, encoding = encode_classes(y)
print(y_enc, '\n', encoding)

[0 0 1 0 0 0 0 0 0 0 1 1 0 1 1 1 1 1 1 0 1 0 1 0 1 1 0 0 1 0 1 0 0 0 0 1 1
 1 0 1 0 1 0 1 0 0 1 0 1 1 0 1 0 1 1 0 1 1 1 1 0 1 1 0 1 1 0 0 1 1 0 0 0 1
 0 1 1 1 0 1 0 0 1 0 1 1 0 1 1 1 1 1 0 1 1 0 1 0 0 1 1 1 1 1 1 1 1 1 0 0 1
 1 0 0 0 1 0 1 1 0 1 1 0 1 0 1 0 0 1 0 0 0 1 0 1 0 0 1 1 1 0 1 1 1 0 0 0 0
 1 0 1 1 1 1 0 0 1 0 1 0 1 1 1 1 1 1 0 1 0 1 1 0 1 0 0 1 0 1 0 0 1 1 0 1 1
 0 0 0 1 0 0 0 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1] 
 {'R': 0, 'M': 1}


Removing features with low impact on target class

In [638]:
corr_arr = []
for idx, row in enumerate(x_norm):
    arr = np.asarray(list(row) + [y_enc[idx]])
    corr_arr.append(arr)
corr_arr = np.asarray(corr_arr)
corr_map = np.corrcoef(corr_arr, rowvar=False).round(2)
corr_map = corr_map[:, 60]  # keep only final column of the heatmap | correlation to target class
corr_map = corr_map.reshape((61, 1))

to_drop = []
for idx, value in enumerate(corr_map):
    if value > -0.1 and value < 0.1:
        to_drop.append(idx)
to_drop

x_norm = np.delete(x_norm, to_drop, axis=1)
x_norm.shape

(208, 37)

Splitting into train, val and test sets

In [639]:
np.array(np.unique(y_enc, return_counts=True)).T

array([[  0,  97],
       [  1, 111]], dtype=int64)

In [640]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test  = train_test_split(x_norm, y_enc, test_size=0.1)
x_train, x_val, y_train, y_val =  train_test_split(x_train, y_train, test_size=0.3)

In [641]:
print(x_train.shape, x_val.shape, x_test.shape)
print(y_train.shape, y_val.shape, y_test.shape)

(130, 37) (57, 37) (21, 37)
(130,) (57,) (21,)


In [642]:
np.array(np.unique(y_train, return_counts=True)).T

array([[ 0, 64],
       [ 1, 66]], dtype=int64)

In [643]:
np.array(np.unique(y_val, return_counts=True)).T

array([[ 0, 25],
       [ 1, 32]], dtype=int64)

In [644]:
np.array(np.unique(y_test, return_counts=True)).T

array([[ 0,  8],
       [ 1, 13]], dtype=int64)

Creating model in Keras (tensorflow)

In [645]:
from keras.models import Model
from keras.layers import Input, Dense
from keras.activations import sigmoid, relu, hard_sigmoid, tanh, softmax

input = Input(shape=(x_norm.shape[1], ), name="input")
dense1 = Dense(128, activation=lambda x: relu(x, alpha=0.3))(input,)
dense2 = Dense(128, activation=lambda x: relu(x, alpha=0.3))(dense1)
dense3 = Dense(128, activation=lambda x: relu(x, alpha=0.3))(dense2)
dense4 = Dense(128, activation=lambda x: relu(x, alpha=0.3))(dense3)
dense5 = Dense(64, activation=lambda x: relu(x, alpha=0.3))(dense4)
dense6 = Dense(64, activation=lambda x: relu(x, alpha=0.3))(dense5)
dense7 = Dense(64, activation=lambda x: relu(x, alpha=0.3))(dense6)
dense8 = Dense(64, activation=lambda x: relu(x, alpha=0.3))(dense7)
dense_out = Dense(1, activation=sigmoid, name="dense_out")(dense8)
model = Model(input, dense_out, name='test_model')

In [646]:
from keras.optimizers import Adam, SGD, RMSprop
from keras.losses import BinaryCrossentropy
from keras.metrics import BinaryAccuracy, AUC

optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.99)
model.compile(loss=BinaryCrossentropy(from_logits=False), optimizer=optimizer, metrics=[BinaryAccuracy(), AUC()])

In [647]:
val_data = (x_val, y_val)
model.fit(x=x_train, y=y_train, validation_data=val_data, batch_size=5, epochs=25, verbose=1)

Epoch 1/25
26/26 [==============================] - 1s 8ms/step - loss: 0.6634 - binary_accuracy: 0.5846 - auc_31: 0.6481 - val_loss: 0.6138 - val_binary_accuracy: 0.6491 - val_auc_31: 0.7119
Epoch 2/25
26/26 [==============================] - 0s 3ms/step - loss: 0.5312 - binary_accuracy: 0.7385 - auc_31: 0.8086 - val_loss: 0.5382 - val_binary_accuracy: 0.7368 - val_auc_31: 0.7856
Epoch 3/25
26/26 [==============================] - 0s 3ms/step - loss: 0.4338 - binary_accuracy: 0.8154 - auc_31: 0.8729 - val_loss: 0.4929 - val_binary_accuracy: 0.7719 - val_auc_31: 0.8281
Epoch 4/25
26/26 [==============================] - 0s 3ms/step - loss: 0.4312 - binary_accuracy: 0.8308 - auc_31: 0.8673 - val_loss: 0.5031 - val_binary_accuracy: 0.7368 - val_auc_31: 0.8356
Epoch 5/25
26/26 [==============================] - 0s 3ms/step - loss: 0.4047 - binary_accuracy: 0.8462 - auc_31: 0.8955 - val_loss: 0.5521 - val_binary_accuracy: 0.7368 - val_auc_31: 0.8275
Epoch 6/25
26/26 [======================

In [648]:
model.evaluate(x=x_test, y=y_test)

1/1 [==============================] - 0s 3ms/step - loss: 0.4759 - binary_accuracy: 0.8095 - auc_31: 0.9760


[0.475899875164032, 0.8095238208770752, 0.9759615659713745]

In [649]:
model.summary()

Model: "test_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 37)]              0         
                                                                 
 dense_227 (Dense)           (None, 128)               4864      
                                                                 
 dense_228 (Dense)           (None, 128)               16512     
                                                                 
 dense_229 (Dense)           (None, 128)               16512     
                                                                 
 dense_230 (Dense)           (None, 128)               16512     
                                                                 
 dense_231 (Dense)           (None, 64)                8256      
                                                                 
 dense_232 (Dense)           (None, 64)                4